# Расширение данных (Data augmentation)

Для улучшения качества нашей модели, мы можем увеличить количество данных.

В данном примере мы сгенерируем данные из уже существующих.

Например:
- отразить изображение по вертикали и горизонтали
- повернуть изображение
- немного увеличить или уменьшить масштаб
- изменить изображение другими шумами